# Relevant Packages

In [1]:
#Import packages
import IPython.display as ipd
import numpy as np
import pyaudio, wave
import math
import matplotlib.pyplot as plt
from numpy.matlib import repmat
from scipy.fftpack import dct

# MFCC Class

In [2]:
import numpy as np
from scipy.fftpack import dct
import math
import matplotlib.pyplot as plt
from numpy.matlib import repmat
class MFCC:
    def __init__(self,overlap_time=0.01,segment_time=0.02,window_mode="Hamming",Mel_setting=None,normalize_flag=False):
        self.segment_time=segment_time
        self.overlap_time=overlap_time
        self.Mel_setting=Mel_setting
        self.lower_boundHz=0
        self.upper_boundHz=8000
        self.num_banks=40
        self.num_ceps=13
        self.mode=window_mode
        self.normalize=normalize_flag
        if self.Mel_setting !=None:
            self.lower_boundHz=self.Mel_setting["lower_boundHz"]
            self.upper_boundHz=self.Mel_setting["upper_boundHz"]
            self.num_banks=self.Mel_setting["num_banks"]
            self.num_ceps=self.Mel_setting["num_ceps"]
    #Apply window function to the frames
    def wave2frames(self):
        width=int(self.rate*self.segment_time)
        stride=int(self.rate*self.overlap_time)
        # you can use np.ceil(log2 width) as the power of FFT 
        self.FFT_size=int(np.power(2,np.ceil(np.log2(width))))
        wave_length=len(self.wave)
        if wave_length<=width:
            num_frames=1
        else:
            num_frames=int(np.ceil((wave_length-width)/stride))+1
        self.frames=[]    
        for i in range(num_frames):
            self.frames.append(self.wave[stride*i:stride*i+width])
        
        
    def Preemphasizing(self):
        #input is the chunk with length set to 1600, it could also be the whole signal document (preferred)
        pe=np.zeros(len(self.wave))
        pe[0]=self.wave[0]
        for i in range(1,len(self.wave)):
            pe[i]=self.wave[i]-0.95*self.wave[i-1]
        self.wave=pe
        
        
    def windowing(self,mode="Hamming"):
        #do windowing for the signal frame
        n=len(self.frames[0])
        #windowing the speech signal
        if self.mode=="Hamming":
            self.frames=self.frames*np.hamming(n)
        elif self.mode=="Hanning":
            self.frames=self.frames*np.hanning(n)
        elif self.mode=="Blackman":
            self.frames=self.frames*np.blackman(n)


    #Pad zero to make the frame a desired length (exponential of 2)
    def zero_padding(self):
        #zero padding for FFT
        frame_length=len(self.frames[0])
        num_frames=len(self.frames)
        padding_length=self.FFT_size-frame_length
        zeros=np.zeros([num_frames,padding_length])
        print(frame_length)
        print(num_frames)
        print(zeros.shape)
        print(np.array(self.frames).shape)
        self.frames=np.concatenate((self.frames,zeros),axis=1)
        print(self.frames.shape)

    #Fast-Fourier Transform
    def FFT(self):
        #http://dong.sh/posts/difffftandrfft/
        #we use the rfft rather than the fft, the difference is shown in the website above.
        # in short rfft give a result of half of the self.FFT_size
        mag_frames = np.absolute(np.fft.rfft(self.frames, self.FFT_size))  # Magnitude of the FFT
        self.power_spectrum = ((1.0 / self.FFT_size) * ((mag_frames) ** 2))  # Power Spectrum
        print(self.power_spectrum.shape)

    #Herz to Mel
    def hz2mel(self, f):
        #  Convert frequencies f (in Hz) to mel 'scale'.
        #Mel warping function
        z = 2595 * np.log10(1+f/700);
        return z

    #Mel to Herz
    def mel2hz(self, z):
        # Convert 'mel scale' frequencies into Hz
        f = 700*(np.power(10,z/2595)-1)  #CHR use np.power to simulate ".^" in matlab
        return f

    
                                                             
                                                             
    def filterbanks(self):
        upper_boundMel=self.hz2mel(self.upper_boundHz)
        lower_boundMel=self.hz2mel(self.lower_boundHz)
        #Uniformly sample in Mel scale to get 40 triangular filters
        mel_points=np.linspace(lower_boundMel,upper_boundMel,self.num_banks+2)
        #Transform to frequency scale and generate non-uniformly distributed triangular filters
        hz_points=self.mel2hz(mel_points)
        #After FFT, the sample become half of the original (integer)
        bin=np.floor((self.FFT_size + 1)/2*hz_points/(self.upper_boundHz-self.lower_boundHz))
        #Correspond the frequency points to the power spectrum after FFT
       #print(bin)
        #Normalization because of the lower bound
        bin=bin-bin[0]
        fbank=np.zeros((self.num_banks,int(np.floor(self.FFT_size/2+1))))
        for m in range(1,self.num_banks+1):
            f_m_low=int(bin[m-1])   # left
            f_m_center=int(bin[m])  # center
            f_m_up=int(bin[m+1])    # right
            for k in range(f_m_low,f_m_center):
                fbank[m-1,k]=(k-bin[m-1])/(bin[m]-bin[m-1])
            for k in range(f_m_center,f_m_up):
                fbank[m-1,k]=(bin[m+1]-k)/(bin[m+1]-bin[m])
        #Dot product
        Fb=np.dot(self.power_spectrum,fbank.T)
        #eps is a small float number. when Fb is 0, Fb is eps. this is to avoid error in division
        Fb=np.where(Fb==0,np.finfo(float).eps,Fb) # Numerical Stability

        #Log Mel spectrum
        #Compress values
        self.Log_Mel_spectrum=10*np.log(Fb) 

    def cal_delta(self):
        self.mfcc[0]=np.hstack((self.mfcc[0],self.mfcc[0],self.mfcc[0]))
        self.mfcc[-1]=np.hstack((self.mfcc[-1],self.mfcc[-1],self.mfcc[-1]))

        for i in range(1,len(self.mfcc)-1):
            delta=np.zeros(13)
            for j in range(13):
                delta[j]=self.mfcc[i+1][j]-self.mfcc[i-1][j]
            self.mfcc[i]=np.hstack((self.mfcc[i],delta))

        for i in range(1,len(self.mfcc)-1):
            acc=np.zeros(13)
            for j in range(13):
                acc[j]=self.mfcc[i+1][13+j]-self.mfcc[i-1][13+j]
            self.mfcc[i]=np.hstack((self.mfcc[i],acc))
                
    def normalization(self):
        #对mfcc_feat 归一化
        mean, std = np.mean(self.mfcc), np.std(self.mfcc)
        self.mfcc = (self.mfcc-mean) / std
    
    def getMFCC(self,wave,rate=16000):
        self.wave=wave
        self.rate=rate
        self.Preemphasizing()
        self.wave2frames()
        self.windowing()
        self.zero_padding()
        self.FFT()
        self.filterbanks()
        self.mfcc=dct(self.Log_Mel_spectrum, type=2,axis=1, norm='ortho')[:,0 : 13].tolist()
        self.cal_delta()
        if self.normalize:
            self.normalization()
            
        return self.mfcc

# MFCC PART

Here, I write three functions to get MFCC feature (39 dim) one is from project 2 another two functions are made of librosa and python_speech_features. 

In [3]:
def getMFCC(wavename):
    # It runs tooo slow
    import numpy as np
    import scipy.io.wavfile as wav
    fs, audio = wav.read(wavename)
    my_mfcc=MFCC(normalize_flag=True)
    mfccs=my_mfcc.getMFCC(audio,fs)
    return np.array(mfccs)    

In [4]:
def getMFCC2(wavename):#with normalization
    import numpy as np
    import scipy.io.wavfile as wav
    from python_speech_features import mfcc
    fs, audio = wav.read(wavename)
    feature_mfcc = mfcc(audio, samplerate=fs,
                        winlen=0.020,
               winstep=0.01,
               numcep=13,
               nfilt=40,
               nfft=512,
               lowfreq=133.33,
               highfreq=6855.4976,
               preemph=0.95,
               ceplifter=0,
               appendEnergy=False,
               winfunc=np.hamming)
    mfcc=[]
    mfcc.append(np.hstack([feature_mfcc[0],feature_mfcc[0],feature_mfcc[0]]))
    for i in range(1,len(feature_mfcc)-1):
        delta=np.zeros(13)
        for j in range(13):
            delta[j]=feature_mfcc[i+1][j]-feature_mfcc[i-1][j]
        mfcc.append(np.hstack([feature_mfcc[i],delta]))
    mfcc.append(np.hstack([feature_mfcc[-1],feature_mfcc[-1],feature_mfcc[-1]]))

    for i in range(1,len(mfcc)-1):
        acc=np.zeros(13)
        for j in range(13):
            acc[j]=mfcc[i+1][13+j]-mfcc[i-1][13+j]
        mfcc[i]=np.hstack([mfcc[i],acc])
    mfccs=np.array(mfcc)
    std=np.std(mfccs)
    var=np.var(mfccs,1)
    for i in range(len(mfccs)):
        for j in range(39):
            mfccs[i][j]=mfccs[i][j]/var[i]
    return mfccs

In [5]:
def getMFCC3(wavename):#with normalization
    import librosa
    audio, sr = librosa.load(wavename,sr=None)
    mfccs = librosa.feature.mfcc(y=audio, sr=sr,n_mfcc=39)
    std=np.std(mfccs)
    var=np.var(mfccs,1)
    for i in range(len(mfccs)):
        for j in range(39):
            mfccs[i][j]=mfccs[i][j]/var[i]
    return mfccs

# Create Templates


There are total 100 wave files, 10 samples for digit 1-10. All these files are saved in the following format by Huangrui Chu: digit_0.wav, digit_1.wav.  digit could be 0,1,2,...9.

In [6]:
def create_templates(foldername,start_instance,end_instance):
    #we could also use this function for project 1,2,3 for testing our algorithm
    #INPUT:
        #foldername: the name of folder where Huangrui save the audio files
        #end-start:the number of instance we are expected to use as the templates.
    #OUTPUT:
        #templates: we can easily fetch the template for the corresponding digit using:
                #len(templates)%10+digit
    templates=[]
    max_template_length=0
    for i_th_instance in range(start_instance,end_instance):
        for digit in range(0,10):
            #print(str(digit )+'_'+str(i_th_instance)+'.wav')
            mfcc=getMFCC2(foldername+"/"+str(digit )+'_'+str(i_th_instance)+'.wav')
            if max_template_length<len(mfcc):
                max_template_length=len(mfcc)
            templates.append(mfcc)
    return templates,max_template_length

In [18]:
def create_templates2(foldername,start_instance,end_instance):
    #we could also use this function for project 1,2,3 for testing our algorithm
    #INPUT:
        #foldername: the name of folder where Huangrui save the audio files
        #end-start:the number of instance we are expected to use as the templates.
    #OUTPUT:
        #templates: we can easily fetch the template for the corresponding digit using:
                #len(templates)%10+digit
    digits=["zero","one","two","three","four","five","six","seven","eight","nine"]
    templates=[]
    max_template_length=0
    for i_th_instance in range(start_instance,end_instance):
        for digit in digits:
            #print(str(digit )+'_'+str(i_th_instance)+'.wav')
            mfcc=getMFCC2(foldername+"/"+digit+'_'+str(i_th_instance)+'.wav')
            if max_template_length<len(mfcc):
                max_template_length=len(mfcc)
            templates.append(mfcc)
    return templates,max_template_length

# Problem 2 K Means Hard Version

Use the segmental K-means procedure to train an HMM for each of the digits
(using the 5 "training" recordings you have for them). Assume each state to
have a single Gaussian distribution, and the HMM for each digit to have 5
states. Recognize the 5 test utterances using the HMM models and report
recognition accuracy.

In [8]:
 def traceback(D):
    #start from the last state and last frame
    current_state,current_frame=np.array(D.shape)-1
    #insert the last frame's state
    x=[current_state]
    
    #print(current_frame+1)
    # we do not need the frame 0, which is the fine
    while current_state>0 and current_frame>1:
        #move to the previous frame
        current_frame-=1
        #print(current_state)
        if current_state>2:
            to_check=[D[current_state][current_frame-1],
                      D[current_state-1][current_frame-1],
                      D[current_state-2][current_frame-1]]
            track=np.argmin(to_check)
        elif current_state>1:
            to_check=[D[current_state][current_frame-1],
                      D[current_state-1][current_frame-1]]
            track=np.argmin(to_check)
        else:
            track=0
            
        if track==0:
            #which means, last frame still in the same stage
            x.insert(0,current_state)
        elif track==1:
            current_state-=1
            x.insert(0,current_state)
        else:
            current_state-=2
            x.insert(0,current_state)
    #print(x)
    
#     print(len(x))
#    print(np.bincount(x, weights=None, minlength=0))
#     print()
    return x

In [9]:
def log_gaussian(mu,squared_sigma,input_vector):
    #Author: Huangrui Chu, 
    #Calculate the cost using log gaussian
    part1=0.5*np.sum(np.log((2*np.pi)*(squared_sigma)))
    part2=0.5*np.sum(np.square((input_vector-mu))/squared_sigma)
    cost= part1+part2
    return cost

In [10]:
import numpy as np
def Kmean_dtw(model_vectors,data,T,cov_cent,get_track=False):
    #Author: Huangrui Chu
    #Input, T is the transition_score
    #yes, we use cost!!!!!!!!
    #Output: the path, how we align each node
    # insert fin at the beginning of the template and data
    zeros=np.zeros([39])
    model_vectors=np.vstack([zeros,model_vectors])
    #这边对应的是不是我的cov 要append1？？？???????????????????????????????????
    ones=np.zeros([39])+1
    cov_cent=np.vstack([ones,cov_cent])
    data=np.vstack([zeros,data])
    #print(data.shape)

    t=len(model_vectors) # here, t should be the number of states+1 we set
    d=len(data)#means input frame j
    #create empty best path cost matrix "P" 
    P=np.zeros([t,d])
    #to fetch the data, we use P[i][j],i for template and j for input data
#     • P i,j = best path cost from origin to node [i,j]
#     • C i,j = local node cost of aligning template frame i to input frame j
#     • T i,j,k,l = Edge cost from node (i,j) to node (k,l)

    for j in range(0,d): #input frame j
        for i in range(t): # i th template frame aligns with j-th input frame
            #6 PPT p.g. 65
            Cij= log_gaussian(model_vectors[i],cov_cent[i],data[j])
            #print(Cij)
            if i-2>=0:
                P[i][j]=min(P[i][j-1]+T[i][i],P[i-1][j-1]+T[i-1][i],
                            P[i-2][j-1]+T[i-2][i])+Cij
            elif i-1>=0:
                P[i][j]=min(P[i][j-1]+T[i][i],P[i-1][j-1]+T[i-1][i])+Cij
            else:
                P[i][j]=P[i][j]+Cij
                
    #Use DTW cost / frame of input speech, instead of total DTW cost, before determining threshold
    # 5 PPT  p.g 32
    P=P/d
#     print(P.shape)
#     print(P[-1][-1])
#     print(P)
    distance=P[-1][-1]
    if get_track:
        return distance,traceback(P)
    else:
        return distance

In [11]:
import numpy as np
def Kmeans(templates_for_Kmeans,state_number=5):
    def fetch_mvectors_and_cov(templates_for_Kmeans,node_state,state_number):
        model_vectors=[]#model vector of each state
        cov_cent=[]#covariance of each state
        node_in_each_state=[]#state number decide sublist number
        # node_in_each_state[0] is empty
        for state in range(state_number+1):
            node_in_each_state.append([])
            
        for k in range(len(templates_for_Kmeans)):#templates number
            # the i th vector of the k th training sequence
            for i in range(len(node_state[k])):
                j=int(node_state[k][i])#the state of the i th vector
                node_in_each_state[j].append(templates_for_Kmeans[k][i])

        for j in range(1,state_number+1):
            #calculate the initial mean of all the vectors belong to state j
            #print(node_in_each_state[j])
            m_j=np.mean(node_in_each_state[j],0)
            model_vectors.append(m_j)
            #transform the list to array
            node_in_each_state[j]=np.array(node_in_each_state[j])
            #calculate the initial cov of all the vectors belong to state j
            cov_temp=np.cov(np.array(node_in_each_state[j]).T)
            cov_diagnol=np.diagonal(cov_temp, offset=0, axis1=0, axis2=1)
            cov_cent.append(cov_diagnol)
            #print(cov_diagnol)
        #print("yeah~~,CHR is soooo good")
        return model_vectors,cov_cent
    
    def compute_transition_cost(node_state,state_number):
        # 6 PPT p.g 53
        #return the transition probability matrix
        #have five states+fine(which means Huangrui Chu set 6 states here)
        shift_likehood=np.zeros((state_number+1,state_number+1))
        state_node_num=np.zeros(state_number+1)
        #fetch all the initial state
        initial_states=[]
        for k in range(len(node_state)):
            shift_likehood[0][node_state[k][0]]+=1
            
        # count the state transition of all the nodes
        for k in range(len(node_state)):
            for i in range(len(node_state[k])-1):
                current_node=node_state[k][i]
                next_node=node_state[k][i+1]
                shift_likehood[current_node][next_node]+=1 
                state_node_num[current_node]+=1
            #last node case        
            shift_likehood[node_state[k][-2]][node_state[k][-1]]+=1
            state_node_num[node_state[k][-1]]+=1
        #It is sometimes useful to permit entry directly into later states
        for j in range(state_number+1):
            #N is the total number of training sequences
            N=len(node_state)
            #N_0j is the number of training sequences for which
            #the first data vector was in the j th state
            N_0j=shift_likehood[0][j]
            shift_likehood[0][j]=N_0j/N
            if N_0j==0:
                shift_likehood[0][j]=np.inf
            else:
                shift_likehood[0][j]=-np.log(shift_likehood[0][j])
            
        #6 PPT p.g 55
        for j in range(1,state_number+1):
            for k in range(j,state_number+1):
                shift_likehood[j][k]=shift_likehood[j][k]/state_node_num[j]
                #transition probability---->transition cost
                #T_ij in 6 PPT p.g 58
                if shift_likehood[j][k]!=0:
                    shift_likehood[j][k]=-np.log(shift_likehood[j][k])
                else:
                    shift_likehood[j][k]=np.inf
        transition_cost=np.array(shift_likehood)
        #print("likelihood is")
        #print(transition_cost)
        return transition_cost
    
    #initial setup
    node_state=[]#each node in which state, all templates together
    for k in range(len(templates_for_Kmeans)):#templates number
        #now, it is the k th training sequence we are look at
        #for initial part, each state have even number of nodes
        n_node=len(templates_for_Kmeans[k])//state_number
        num_left_nodes=len(templates_for_Kmeans[k])%state_number
        #n_node is the N_kj
        # to store the node state of k th sequence
        current_sample_node_state=np.zeros(len(templates_for_Kmeans[k])).astype(int) 
        #now, initialize the node state for each node in the k th sequence
        for i in range(1,state_number+1):
            current_sample_node_state[n_node*(i-1):n_node*i]= int(i)
        #left nodes be the last state:
        if num_left_nodes!=0:
            current_sample_node_state[-num_left_nodes:]= int(state_number)
        node_state.append(current_sample_node_state)
        #to check my initial node state assignment
        #print(node_state[k])
        #print(np.bincount(node_state[k], weights=None, minlength=0))
        
    # By now, for every training squence, we fetch the initial node state
    #Next, we need to calculate the model_vectors and cov_cent
    model_vectors,covarience=fetch_mvectors_and_cov(templates_for_Kmeans,
                                                         node_state,
                                                         state_number)
    #Next,to simulate the transition structures, we need to calculate the
    #transition probability
    transition_cost=compute_transition_cost(node_state,state_number)
    
    #complete set up by iteratively update the model vectors,covariance
    #and transition score
    previous_best_distance=-np.inf
    current_best_distance=0
    for j in range(1,100):
        #update the node state
        for k in range(len(templates_for_Kmeans)):
#             print("previous")
#             previous_state_count=np.bincount(node_state[k], weights=None, minlength=0)
            distance,node_state[k]=Kmean_dtw(model_vectors,templates_for_Kmeans[k]
                                    ,transition_cost,covarience,get_track=True)
            current_best_distance+=distance
#             print("later")
#             later_state_count=np.bincount(node_state[k], weights=None, minlength=0)
            
            
        #according to the updated node state, update model_vectors,covarience
        model_vectors,covarience=fetch_mvectors_and_cov(templates_for_Kmeans,
                                                             node_state,
                                                             state_number)
        transition_cost=compute_transition_cost(node_state,state_number)
        #Convergence is achieved when the total best-alignment error for
        #all training sequences does not change significantly with further
        #refinement of the model
        difference= previous_best_distance-current_best_distance
#         print("current difference")
#         print(difference)
        previous_best_distance=current_best_distance
#         print("updated previous best distance")
#         print(previous_best_distance)
        current_best_distance=0
        if abs(difference)<0.0015:
            print("Use {} iterators to updata".format(j))
            break
        
    return model_vectors,transition_cost,covarience

In [12]:
def create_Kmeans_templates(foldername,start_instance,end_instance):
    #we could also use this function for project 1,2,3 for testing our algorithm
    #INPUT:
        #foldername: the name of folder where Huangrui save the audio files
        #end-start:the number of instance we are expected to use as the templates.
    #OUTPUT:
        #templates: we can easily fetch the template for the corresponding digit using:
                #len(templates)%10+digit
    templates_models=[]
    templates_likehood=[]
    templates_cov=[]
    
    for digit in range(0,10):
        print("Huangrui is creating model vector for digit {} using {} sequences" .format(
            digit,end_instance-start_instance))
        templates=[]
        for i_th_instance in range(start_instance,end_instance):
            #print(str(digit )+'_'+str(i_th_instance)+'.wav')
            mfcc=getMFCC2(foldername+"/"+str(digit )+'_'+str(i_th_instance)+'.wav')
            templates.append(mfcc)
        model,likehood,cov=Kmeans(templates)
        #print(model)
        templates_models.append(model)
        templates_likehood.append(likehood)
        templates_cov.append(cov)
 
    return templates_models,templates_likehood,templates_cov

In [13]:
digits=["zero","one","two","three","four","five","six","seven","eight","nine"]

In [14]:
def create_Kmeans_templates2(foldername,start_instance,end_instance):
    #we could also use this function for project 1,2,3 for testing our algorithm
    #INPUT:
        #foldername: the name of folder where Huangrui save the audio files
        #end-start:the number of instance we are expected to use as the templates.
    #OUTPUT:
        #templates: we can easily fetch the template for the corresponding digit using:
                #len(templates)%10+digit
    templates_models=[]
    templates_likehood=[]
    templates_cov=[]
    digits=["zero","one","two","three","four","five","six","seven","eight","nine"]
    for digit in digits:
        print("Huangrui is creating model vector for digit {} using {} sequences" .format(
            digit,end_instance-start_instance))
        templates=[]
        for i_th_instance in range(start_instance,end_instance):
            #print(str(digit )+'_'+str(i_th_instance)+'.wav')
            mfcc=getMFCC2(foldername+"/"+digit+'_'+str(i_th_instance)+'.wav')
            templates.append(mfcc)
        model,likehood,cov=Kmeans(templates)
        #print(model)
        templates_models.append(model)
        templates_likehood.append(likehood)
        templates_cov.append(cov)
 
    return templates_models,templates_likehood,templates_cov

In [16]:
foldername="../data"
foldername2="../manjunkaidata/templates"
templates_models,templates_likehood,templates_cov=create_Kmeans_templates2(foldername2,1,6)

Huangrui is creating model vector for digit zero using 5 sequences
Use 14 iterators to updata
Huangrui is creating model vector for digit one using 5 sequences
Use 15 iterators to updata
Huangrui is creating model vector for digit two using 5 sequences
Use 11 iterators to updata
Huangrui is creating model vector for digit three using 5 sequences
Use 10 iterators to updata
Huangrui is creating model vector for digit four using 5 sequences
Use 9 iterators to updata
Huangrui is creating model vector for digit five using 5 sequences
Use 11 iterators to updata
Huangrui is creating model vector for digit six using 5 sequences
Use 7 iterators to updata
Huangrui is creating model vector for digit seven using 5 sequences
Use 6 iterators to updata
Huangrui is creating model vector for digit eight using 5 sequences
Use 7 iterators to updata
Huangrui is creating model vector for digit nine using 5 sequences
Use 7 iterators to updata


In [19]:
foldername3="../manjunkaidata/queries"
test_data,max_length=create_templates2(foldername3,1,6)

In [20]:
def getKMeans_HMM_accuracy(templates_models,test_data,templates_likehood,templates_cov):
    #K-means HMM recognition
    import numpy as np

    #main
    accuracy=0
    for j in range(len(test_data)):
        current_digit=j%10
        smallest_distance=[1000000,0]
        for i in range(10):
            distance=Kmean_dtw(templates_models[i],test_data[j],templates_likehood[i],templates_cov[i])
            if distance<smallest_distance[0]:
                smallest_distance[0]=distance
                smallest_distance[1]=i
        recoginized_result=smallest_distance[1]
        if current_digit==recoginized_result:
            accuracy+=1
        print(current_digit,'recognized as:',recoginized_result)
    print('accuracy:',accuracy/len(test_data))

In [21]:
getKMeans_HMM_accuracy(templates_models,test_data,templates_likehood,templates_cov)

0 recognized as: 0
1 recognized as: 1
2 recognized as: 2
3 recognized as: 3
4 recognized as: 4
5 recognized as: 5
6 recognized as: 6
7 recognized as: 7
8 recognized as: 8
9 recognized as: 1
0 recognized as: 0
1 recognized as: 1
2 recognized as: 2
3 recognized as: 3
4 recognized as: 4
5 recognized as: 5
6 recognized as: 6
7 recognized as: 1
8 recognized as: 8
9 recognized as: 9
0 recognized as: 0
1 recognized as: 1
2 recognized as: 2
3 recognized as: 3
4 recognized as: 4
5 recognized as: 5
6 recognized as: 5
7 recognized as: 7
8 recognized as: 8
9 recognized as: 1
0 recognized as: 0
1 recognized as: 1
2 recognized as: 2
3 recognized as: 3
4 recognized as: 4
5 recognized as: 5
6 recognized as: 6
7 recognized as: 1
8 recognized as: 8
9 recognized as: 9
0 recognized as: 0
1 recognized as: 1
2 recognized as: 2
3 recognized as: 3
4 recognized as: 4
5 recognized as: 5
6 recognized as: 6
7 recognized as: 7
8 recognized as: 8
9 recognized as: 1
accuracy: 0.88
